# imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install unidecode
!pip install emoji

import re
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from string import digits
import json
from datetime import datetime

import math
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk import download as nltk_download
from emoji import demojize
import emoji
import unidecode as ud
from nltk import bigrams

warnings.filterwarnings('ignore')
nltk_download('stopwords')
nltk_download('punkt')
nltk_download('wordnet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 4.2 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# data gathering

In [ ]:
path = 'drive/MyDrive/crypto-classifier/data'
d1=pd.read_csv(f'{path}/new_tag/Darya.v1.csv', encoding='UTF-8',dtype='str')
d2=pd.read_csv(f'{path}/tag3/Nargess.v2.csv', encoding='UTF-8',dtype='str')
d3=pd.read_csv(f'{path}/tag3/Hana.v2.csv', encoding='UTF-8',dtype='str')
d4=pd.read_csv(f'{path}/multy_platform/instagram_tags_1.csv', encoding='UTF-8',dtype='str')
d5=pd.read_csv(f'{path}/multy_platform/Reddit_3000_Data_tagged_1.csv', encoding='UTF-8',dtype='str')

data=pd.concat([d4.rename(columns={'Polarity'	:'polarity'}),d5]).reset_index(drop=True)
data['polarity']=data['polarity'].astype(float).astype(int)
labels = {0:'not positive', 1:'positive'}
for i in range(len(data['polarity'])):
  data['polarity'][i]=labels[data['polarity'][i]]
data=data[['text','polarity']]

In [ ]:
path = 'drive/MyDrive/crypto-classifier/data'
data=pd.read_csv(f'{path}/tag3/test.csv', encoding='UTF-8',dtype='str')

In [ ]:
# data=data[data['ns'].isna()].reset_index(drop=True)
data=data.dropna(subset=['concat_fields']).reset_index(drop=True)
# data

In [ ]:
# make Uniformity labels
data['signal'] = data['signal'].fillna('no signal').apply(lambda x:x.lower())
data['polarity'] = data['polarity'].fillna('neutral').apply(lambda x:x.lower())

data['signal']=data['signal'].apply(lambda x:'no signal' if x in ['no siganl','no signale','not buy','-','bno signal'] else x)
data['signal']=data['signal'].apply(lambda x:'hold' if x in ['bhold','keep'] else x)
data['signal']=data['signal'].apply(lambda x:'buy' if x in ['nbuy','positie','positive'] else x)
data['polarity']=data['polarity'].apply(lambda x:'neutral' if x in ['nutral','-'] else x)
data['polarity']=data['polarity'].apply(lambda x:'negative' if x in ['negetive','negetive\n'] else x)
data['polarity']=data['polarity'].apply(lambda x:'positive' if x in ['positve','positve','positie','posiitve','postive'] else x)
print('categories: ',Counter(data['signal']))
print('polarity: ',Counter(data['polarity']))

categories:  Counter({'buy': 30, 'no signal': 16, 'sell': 6})
polarity:  Counter({'positive': 34, 'neutral': 10, 'negative': 8})


# columns
['index0', 'index1', 'contributors', 'coordinates', 'created_at',
'display_text_range', 'entities', 'favorite_count', 'favorited',
'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
'in_reply_to_status_id', 'in_reply_to_status_id_str',
'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
'lang', 'place', 'retweet_count', 'retweeted', 'source', 'truncated',
'user', 'extended_entities', 'possibly_sensitive', 'retweeted_status',
'quoted_status', 'quoted_status_id', 'quoted_status_id_str',
'quoted_status_permalink', 'user_sname', 'withheld_copyright',
'withheld_in_countries', 'withheld_scope', 'concat_fields',
'clean_tweet', 'coins', 'coins2', 'coins3', 'signal', 'polarity',
'Ambiguous', 'ns', 'discotion or ad']

---



---



# preprocess

In [ ]:
demojize("that's nice 😂", delimiters=(" ", " "))

"that's nice  face_with_tears_of_joy "

In [ ]:

def extract_emojis(s):
  return [c for c in s if c in emoji.UNICODE_EMOJI['en']]

""" Normalize text """
def normalizeToken(token):
    """return normal tweet with mentions and http url
    Args:
        token ([string]): [a word(a token)]
    Returns:
        [string]: [normalized token]
    """
    if token.startswith("@"):
        return "@user"
    elif token.lower().startswith("http") or token.lower().startswith("www"):
        return "httpurl"
    else:
        return (token)

def normalizeTweets(tweets):
    """function for preprocessing
        - demojize
        - unidecode
        - lower
        - lammetize

    Args:
        tweets ([numpy list]): [list of plane text tweets]

    Returns:
        [numpy list]: [list of normalized tweets]
    """

    f = open(f"{path}/englishstop.txt", "r")
    stop_wl_1=f.read()
    f = open(f"{path}/stop_words_english.txt", "r")
    stop_wl_2=f.read()

    punctuations=r'''"&()*,.:;?[\]^`{|}~'''

    # # replace emojies
    tweets=tweets.apply(lambda x:" ".join(demojize(x, delimiters=(" ", " ")).split()))

    # # delete numbers and punctuations

    tweets = tweets.apply(lambda x: ud.unidecode(x))
    tweets = tweets.apply(lambda x: x.translate(str.maketrans(' ', ' ', punctuations)))
    # #lower
    tweets = tweets.apply(lambda x: x.lower())
    # # normalize
    lemma = WordNetLemmatizer()
    # normTweets = tweets.apply(lambda x: (" ".join([normalizeToken(token) for token in x.split()])))
    normTweets = tweets.apply(lambda x: (" ".join([lemma.lemmatize(token, pos = "v") for token in x.split()])))
    normTweets = normTweets.apply(lambda x: (" ".join([lemma.lemmatize(token, pos = "n") for token in x.split()])))

    # # remove stopwords
    normTweets = normTweets.apply(lambda x: " ".join([word for word in x.split() if word not in stopwords.words("english")]))
    normTweets = normTweets.apply(lambda x: " ".join([word for word in x.split() if word not in stop_wl_1]))
    # normTweets = normTweets.apply(lambda x: " ".join([word for word in x.split() if word not in stop_wl_2]))

    for i in range(len(normTweets)):
        if not len(normTweets[i]):
             normTweets[i]='nan'
    return normTweets

def unitext(tweets):
    dolars = tweets.apply(lambda x:set([ t.replace('$','') for t in x.split() if t.startswith('$')and len(t)>1 ]))

    tweets = tweets.apply(lambda x: (" ".join([normalizeToken(token) for token in x.split()])))
    tweets = tweets.apply(lambda x: re.sub(r"\d+", " a_number ", str(x)))
    ut=[]
    for i in range(len(tweets)):
      ut.append(unicoin(tweets[i],dolars[i]))
    return np.array(ut)

def extract_features(mydata):

  """extract hashta """
  return mydata

def unicoin(tweet,dolar):
  tokens=tweet.split()
  for i,w in enumerate(tokens):
    if w[0]=='$' or w[0]=='#':
      w=w[1:]

    if w in dolar or w in list(coins['coin_name']) or w in list(coins['symbol']):
      tokens[i]='a_coin_name'

  return ' '.join(tokens)

coins=pd.read_csv(f'{path}/Top_100_coinMarketCap.csv')
coins['with$']=coins['symbol'].apply(lambda x:'$'+x+' ')
coins['with#']=coins['symbol'].apply(lambda x:'#'+x+' ')
coins['symbol']=coins['symbol'].apply(lambda x:' '+x.upper()+' ')
coins


,cname,coin_name,symbol,with$,with#
0,1inch network,1inch network,1INCH,$1inch,#1inch
1,aave,aave,AAVE,$aave,#aave
2,algorand,algorand,ALGO,$algo,#algo
3,amp,amp,AMP,$amp,#amp
4,arweave,arweave,AR,$ar,#ar
...,...,...,...,...,...
95,wrapped bitcoin,wrapped bitcoin,WBTC,$wbtc,#wbtc
96,xdc network,xdc network,XDC,$xdc,#xdc
97,xrp,xrp,XRP,$xrp,#xrp
98,yearn.finance,yearn.finance,YFI,$yfi,#yfi


In [ ]:
data['concat_fields'] = data['concat_fields'].apply(lambda x: x.replace('&amp;','and'))
# data['clean_text']=normalizeTweets(data['concat_fields'])
# data['unicode_text_clean']=unitext(data['clean_text'])
data['unicode_text_concat']=unitext(data['concat_fields'])

data=extract_features(data)

In [ ]:
data[['concat_fields','concat_fields']]

,concat_fields,concat_fields
0,Gone but not forgotten. \n\nAs Canada lifts th...,Gone but not forgotten. \n\nAs Canada lifts th...
1,The Genesis Penguins Official Rarity Ranking i...,The Genesis Penguins Official Rarity Ranking i...
2,#Bitcoin erased most of its losses earlier in ...,#Bitcoin erased most of its losses earlier in ...
3,⚡️TOP #DeFi Projects by Social Activity\n24 Fe...,⚡️TOP #DeFi Projects by Social Activity\n24 Fe...
4,Will Russia turn to #bitcoin to evade sanctions?,Will Russia turn to #bitcoin to evade sanctions?
5,💥 $LUNA is on fire today and other #DeFi coins...,💥 $LUNA is on fire today and other #DeFi coins...
6,"1/2 Crazy story,my webmaster escaping from Ukr...","1/2 Crazy story,my webmaster escaping from Ukr..."
7,I like #Bitcoin because it is decentralized an...,I like #Bitcoin because it is decentralized an...
8,✨ 10000$ | Top - 10 ✨\nhttps://t.co/Fg1fjhOdsa...,✨ 10000$ | Top - 10 ✨\nhttps://t.co/Fg1fjhOdsa...
9,People talk or have conversations about $ETH f...,People talk or have conversations about $ETH f...


In [ ]:
data['no_punc_clean_tweet'] = data['clean_text'].apply(lambda x: re.sub('[^a-zA-Z0-9#$]+\s*', ' ',x))
data['no_punc_concat_fields'] = data['concat_fields'].apply(lambda x: re.sub('[^a-zA-Z0-9#$]+\s*', ' ',x))
# data['clean_tweet'] = data['clean_text'].apply(lambda x: x.replace('&amp;','and'))
data['coins'] = data['no_punc_clean_tweet'].apply(lambda x :  [coins['cname'][i] for i in range(len(coins)) if (coins['coin_name'][i]+' ' in x.lower()) or (coins['symbol'][i] in x) or (coins['with$'][i] in x.lower()) or (coins['with#'][i] in x.lower())] )
# print('The number of tweets that has coin names is :',len(clean_main_df))
# clean_main_df

for i in range(len(data['coins'])):
  if ' btc ' in data.iloc[i]['no_punc_clean_tweet'] and 'bitcoin' not in data.iloc[i]['coins'] :
    data.iloc[i]['coins'].append('bitcoin')

# clean_main_df2=data[data['coins'].apply(lambda x:len(x)!=0)]
data['coins2'] = data['no_punc_clean_tweet'].apply(lambda x :  [coins['cname'][i] for i in range(len(coins)) if (' '+coins['coin_name'][i]+' ' in x.lower()) or ('$'+coins['coin_name'][i]+' 'in x.lower()) or ('#'+coins['coin_name'][i]+' ' in x.lower())] )
data['coins3'] = data['no_punc_concat_fields'].apply(lambda x :  [coins['cname'][i] for i in range(len(coins)) if (coins['symbol'][i] in x)                    or (coins['with$'][i] in x.lower())            or (coins['with#'][i] in x.lower())] )

# print('The number of tweets that has coin names is :',len(data))

In [ ]:
data['coins1']='A'
for i in range(len(data)):
  try:
    data['coins1'][i]=  list(set(data['coins2'][i]+data['coins3'][i]))
  except:
    print(i,data['coins2'][i])
data [['coins2','coins3','coins1','coins']]
data['coins']=data['coins1']
data.drop(columns=['coins2','coins3','coins1','no_punc_clean_tweet','no_punc_concat_fields'])

# save

In [ ]:
data.drop(columns=['coins2','coins3','coins1'	,'no_punc_clean_tweet','no_punc_concat_fields']).to_csv(f"{path}/tag3/test.csv",encoding='utf-8',index=False)
# data.to_csv("new_tag/cleanTweet1000.csv",encoding='utf-8',index=False)

In [ ]:
with open(f'{path}/tag3/test.pickle', 'wb') as fp:
    pickle.dump(data.drop(columns=['coins2','coins3','coins1'	,'no_punc_clean_tweet','no_punc_concat_fields']), fp)

# test

In [ ]:
s='Our core philosophy revolves around building &amp; shipping a product which is finished and feature complete. So much, that we keep improving on finished features like consensus. The only which will see the light of day is the best @ElrondNetwork. Our achievements this week👇 https://t.co/DErXw1eDRvA nother week with more improvements and preparations for the #elrondmainnet. Getting closer and closer to a new era for Elrond. https://t.co/78Wu78h1bj'
dd=pd.DataFrame()
dd['tweet']=np.array([s])

dd['clean_text']=normalizeTweets(dd['tweet'])
dd
extract_features(dd)

hashtag extraction done!
mention extraction done!
link extraction done!
tokenizing done!
bigram extraction done!


,tweet,clean_text,hashtags,dolar,tokenied,bigrams
0,Our core philosophy revolves around building &...,core philosophy revolve build ship product fin...,[#elrondmainnet],{},"[core, philosophy, revolve, build, ship, produ...","[core philosophy, philosophy revolve, revolve ..."


In [ ]:
with open ('cleaned_dataset_sell.pickle', 'rb') as fp:
    dataset = pickle.load(fp)
dataset['clean_text']=normalizeTweets(dataset['text'])
dataset=extract_features(dataset)
dataset

FileNotFoundError: ignored

In [ ]:
s=['Current Bitcoin Price: USD $47,124.69 GBP £34,863.12 Euro €41,510.44 #bitcoin #btc $btc #btcusd #btcgbp #btceur #crypto #cryptocurrency','@CoinMarketCap This is a great project for everyone  $100 The best for me#SpikeInu is gearing up! #Spike #ComingSoon #Crypto $Eth $Shib $Doge $Saitama $Luffy $Feg $Kishu $Akita  #SpikeInu #countdown #saitama #FLOKI #BabyDogeCoin #shiba #SAFEMOON #BabyShibaInu https://t.co/vK10U7qJXj']
df=pd.DataFrame()
df['tweet']=np.array(s)
df['clean_text']=normalizeTweets(df['tweet'])
df=extract_features(df)
df.iloc[0]['dolar']

hashtag extraction done!
mention extraction done!
link extraction done!
tokenizing done!
bigram extraction done!


{'4712469', 'btc'}

#old

In [ ]:
""" Normalize text """
def normalizeToken(token):
    """return normal tweet with mentions and http url
    Args:
        token ([string]): [a word(a token)]
    Returns:
        [astring]: [normalized token]
    """
    if token.startswith("@"):
        return "@user"
    elif token.startswith("http") or token.startswith("www"):
        return "httpurl"
    # elif len(token) == 1:
    #     return demojize(token)
    else:
        return (token)

def normalizeTweets(tweets):
    """function for preprocessing

    Args:
        tweets ([numpy list]): [list of plane text tweets]

    Returns:
        [numpy list]: [list of normalized tweets]
    """

    punctuations='''"&'()*+,-.:;<=>?[\]^`{|}~'''
    num_pattern = r'[0-9]'

    # delete numbers and punctuations
    tweets = tweets.apply(lambda x: re.sub(num_pattern, '', x))
    tweets = tweets.apply(lambda x: ud.unidecode(x).translate(str.maketrans('', '', punctuations)))
    #lower
    tweets = tweets.apply(lambda x: x.lower())
    # remove stopwords
    tweets = tweets.apply(lambda x: " ".join([word for word in x.split() if word not in stopwords.words("english")]))
    # normalize
    lemma = WordNetLemmatizer()
    normTweets = tweets.apply(lambda x: (" ".join([normalizeToken(token) for token in x.split()])))
    normTweets = normTweets.apply(lambda x: (" ".join([lemma.lemmatize(token, pos = "v") for token in x.split()])))
    normTweets = normTweets.apply(lambda x: (" ".join([lemma.lemmatize(token, pos = "n") for token in x.split()])))

    normTweets = (normTweets.replace("cannot ", "can not ")
                            .replace("n't ", " n't ")
                            .replace("n 't ", " n't ")
                            .replace("ca n't", "can't")
                            .replace("'m ", " 'm ")
                            .replace("'re ", " 're ")
                            .replace("'s ", " 's ")
                            .replace("'ll ", " 'll ")
                            .replace("'d ", " 'd ")
                            .replace("'ve ", " 've ")
                            .replace('*', ' '))
    for i in range(len(normTweets)):
          if not len(normTweets[i]):
             normTweets[i]='nan'
    normTweets=demojize(normTweets, delimiters=("", ""))
    return normTweets

def extract_features(mydata):
  num_pattern = r'[0-9]'

  """extract hashta """
  mydata['hashtags'] = mydata['clean_text'].str.findall("(#[^#\s]+)")
  mydata['dolar'] = mydata['clean_text'].apply(lambda x:set([ t for t in re.sub(num_pattern, '', x).split() if t.startswith('$')and len(t)>1 ]))
  print('hashtag extraction done!')

  mydata['tokenied'] = mydata['clean_text'].apply(lambda x:x.split(' '))
  print('tokenizing done!')

  mydata['bigrams'] = mydata['tokenied'].apply(lambda x: list(bigrams(x)))
  mydata['bigrams'] = mydata['bigrams'].apply(lambda x: [str(tupl[0] + ' ' + tupl[1]) for tupl in x])
  print('bigram extraction done!')

  return mydata
